In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import decomposition
from sklearn import tree
from sklearn.pipeline import Pipeline
from warnings import simplefilter
import matplotlib.ticker as mtick
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)


results = []
periods = [(10,10),(10,15),(10,20),(15,10),(15,15),(15,20),(20,20),(20,40),(30,60)]


for period in periods:
    
    j = period[0]
    k = period[1]

    features = pd.read_csv('daily_j' + str(j) + '_k' + str(k)+'_features.csv')
    pct = pd.read_csv('daily_j' + str(j) + '_k' + str(k)+'_pct.csv')

    corr = features.corr() 
    #fig, ax = plt.subplots(figsize = (18, 18)) 
    #sns.heatmap(corr[['label']], square=True) 

    correlated_features = set()
    for i in range(len(corr.columns)):
        for a in range(i):
            if abs(corr.iloc[i, a]) > 0.90:
                colname = corr.columns[i]
                correlated_features.add(colname)

    num_colums = ['uint8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numerical_columns = list(features.select_dtypes(include=num_colums).columns)
    df = features[numerical_columns]
    df

    from sklearn.preprocessing import MinMaxScaler
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    pca = decomposition.PCA()
    X_train, X_test, y_train, y_test = train_test_split(df.drop('label',axis=1), 
                                                        df['label'], train_size=0.538, shuffle=False)
    X_before = X_test
    X_test = X_test.dropna()
    X_train = X_train.dropna()

    X_train.drop(columns=correlated_features, axis=1, inplace=True)
    X_test.drop(columns=correlated_features, axis=1, inplace=True)


    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    #y_train.hist()

    dec_tree = tree.DecisionTreeClassifier()
    pipe = Pipeline(steps=[('std_slc', scaler),
                               ('pca', pca),
                               ('dec_tree', dec_tree)])
    n_components = list(range(1,X_train.shape[1]+1,1))

    criterion = ['gini', 'entropy']
    max_depth = [2,4,6,8,10,12]

    parameters = dict(pca__n_components=n_components,
                          dec_tree__criterion=criterion,
                          dec_tree__max_depth=max_depth)

    clf_GS = GridSearchCV(pipe, parameters,n_jobs=-1,verbose=True)
    clf_GS.fit(X_train, y_train)

    print('Best Criterion:', clf_GS.best_estimator_.get_params()['dec_tree__criterion'])
    print('Best max_depth:', clf_GS.best_estimator_.get_params()['dec_tree__max_depth'])
    print('Best Number Of Components:', clf_GS.best_estimator_.get_params()['pca__n_components'])
    print(); print(clf_GS.best_estimator_.get_params()['dec_tree'])


    predictions = clf_GS.predict(X_test)
    predictions

    # scores = cross_val_score(clf_GS, X_train, y_train, cv=10)
    #print('Cross-Validation Accuracy Scores', scores)


    X_before['label'] = predictions.tolist()
    X_before

    performance =pct[['date','index', 'mom_pct_change','rev_pct_change','SP500_pct_change']].infer_objects()
    performance['date'] = pd.to_datetime(performance['date'])
    performance = performance.set_index('date')
    performance = performance.merge(X_before[['label']], left_on='index', right_index=True) #example
    performance['test_pct_change'] = performance.apply(lambda x: x['mom_pct_change'] if x['label']==1 else x['rev_pct_change'] if x['label']==-1 else 0,axis=1) # todo build more models and get their outputs. output results here

    performance[['mom_return','rev_return', 'test_return']] = performance[['mom_pct_change','rev_pct_change', 'test_pct_change']].apply(lambda x: (x+1).cumprod()-1)*100
    
    results.append([j,k,f1_score(y_test,predictions,average='weighted'),predictions,performance['test_return'][-1]])
    
    # import matplotlib.ticker as mtick

    plt = performance[['mom_return','rev_return', 'test_return']].plot(figsize=(15,10),title='J={0} Days, K={1} Days Cummulative Return'.format(j,k),xlabel="Date",ylabel="Percent Return",fontsize=12,color=['green','red','blue'])
    plt.legend(['Momentum','Reversal','Strategy'])
    fmt = '%.0f%%' # Format you want the ticks, e.g. '40%'
    yticks = mtick.FormatStrFormatter(fmt)
    plt.yaxis.set_major_formatter(yticks)
    plt.get_figure().savefig('dt_performance_j{0}_k{1}.png'.format(j,k))

df = pd.DataFrame(results,columns=['j','k','f_score','predictions','test_return']).to_csv('dt_results.csv')



Fitting 5 folds for each of 636 candidates, totalling 3180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 3180 out of 3180 | elapsed:    7.1s finished


Best Criterion: entropy
Best max_depth: 4
Best Number Of Components: 33

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
Fitting 5 folds for each of 684 candidates, totalling 3420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 3420 out of 3420 | elapsed:    6.4s finished


Best Criterion: gini
Best max_depth: 4
Best Number Of Components: 2

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
Fitting 5 folds for each of 744 candidates, totalling 3720 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 3060 out of 3060 | elapsed:    6.5s finished


Best Criterion: entropy
Best max_depth: 8
Best Number Of Components: 24

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
Fitting 5 folds for each of 648 candidates, totalling 3240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 3240 out of 3240 | elapsed:    6.0s finished


Best Criterion: gini
Best max_depth: 2
Best Number Of Components: 26

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
Fitting 5 folds for each of 672 candidates, totalling 3360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    0.8s
